In [1]:
from os.path import join as pjoin
from os import listdir
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import rankdata

from evaluate import get_nan_percent, get_performance, tolerant_mean

In [3]:
bookkeeping_path = ('session_info.csv')

mouse_codes = []
dates = []
versions = []
performances = []
session_indices = np.array([], dtype=int)

for mouse_code in listdir('data'):
    mice_path = pjoin('data', mouse_code)

    cur_mouse_code = []
    cur_date = []
    cur_version = []
    cur_performance = []

    for session in listdir(mice_path):
        date_version = '.'.join(session.split('.')[:-1])
        date = date_version.split('-')[0]
        version = date_version.split('-')[1]
        session_path = pjoin(mice_path, session)
        session_data = pd.read_csv(session_path)
        leftP = session_data['leftP']
        choices = session_data['choices']
        try:
            performance = get_performance(choices=choices, leftP=leftP, version=version)
        except RuntimeError:
            continue
        cur_mouse_code.append(mouse_code)
        cur_date.append(date)
        cur_version.append(version)
        cur_performance.append(performance)
        
    cur_session_indices = rankdata(cur_date).astype(int)
    mouse_codes += cur_mouse_code
    dates += cur_date
    performances += cur_performance
    versions += cur_version
    session_indices = np.concatenate((session_indices, cur_session_indices))

data = pd.DataFrame({'mouse_code': mouse_codes, 
                    'date': dates,
                    'performance': performances,
                    'version': versions,
                    'session_index': session_indices})

data.to_csv(bookkeeping_path, index=False)

choices and set probability has different lengths
choices and set probability has different lengths
choices and set probability has different lengths
choices and set probability has different lengths
choices and set probability has different lengths
choices and set probability has different lengths


In [6]:
data = pd.read_csv('session_info.csv')
filtered = data[data.session_index > 20]
filtered.shape

(377, 5)